In [ ]:
import pandas as pd
import numpy as np
import os 
import pathlib
import PIL
import matplotlib.pyplot as plt
from math import log
from PIL import Image

In [5]:
class MandelbrotSet:
    """Clase para generar y graficar el conjunto de Mandelbrot Z=0 , Z=Z^2+C
       Comprobamos para una matriz de número complejos aquellos valores de C
       en torno a los cuales hay estabilidad"""

    def __init__(self,max_iter :int , threshold : float = 1.0 ):                                 
        self.max_iter = max_iter  
        self.threshold = threshold

    def __contains__(self, c: complex) -> bool:
        """Método mágico que se invoca automáticamente al
           usar la clase junto a operador IN 
           --> Filtra para devolver solo los valores estables"""
        return self.estabilidad(c) == 1


    def estabilidad(self, c: complex, suavizar = False, redondear = True) -> float:  
        """Función para devolver 1 si es estable y el valor de la cuenta si no lo es"""
        contador = self.cuenta(c,suavizar)/self.max_iter 
        return max(0.0, min(contador,1.0)) if redondear else contador
    

    def cuenta(self, c:complex, suavizar = False):
        """Función para calcular la iteración en la que se supera el radio y parar la iteración"""

        z = 0                                                                            
        for i in range(self.max_iter):                                                   
            z=z**2+c
            if abs(z) > self.threshold:
                if suavizar:
                    #Devolvemos versión suavizada con log
                    return i + 1 -log(log(abs(z)))/log(2)    
                #Devolvemos la iteración en la que se ha superado el radio 
                return i 
        #En caso de no superar radio devolvemos iteración máxima      
        return self.max_iter

    
    def evaluar_matriz_compleja(self, x_max:int,y_max:int, escala:float = 0.008):
        """Método para evaluar la estabilidad de los elementos de una matriz 
        comppleja de X_MAX * Y_MAX dimensiones"""
        shape = (x_max, y_max)
        estabilidad = np.zeros(shape)
        matriz = np.zeros(shape,dtype=np.complex_)
        for x in range(x_max):
            for y in range(y_max):
                #generamos la matriz compleja a partir de sus dimensiones con posible escala de grises conforme avanzamos
                c = escala*complex(x, y)   
                estabilidad[x][y] = self.estabilidad(c)
                matriz[x][y] = c.real+1j*c.imag
                #print(c)
        return estabilidad, matriz

    @staticmethod #No hace falta instancias la clase para usar un método estático
    def matriz_compleja(x_max:int,y_max:int, escala:float = 1):
        shape = (x_max, y_max)
        matriz = np.zeros(shape,dtype=np.complex_)
        for x in range(x_max):
            for y in range(y_max):
                #generamos la matriz compleja a partir de sus dimensiones 
                c = escala*complex(x, y)   
                matriz[x][y] = c.real+1j*c.imag
                #print(c)
        return  matriz

    def plot_set(self, width:int = 512, height:int = 512, escala:float = 0.008):
        image = Image.new(mode="L", size=(width,height))

        for y in range(height):
            for x in range(width):
                real = escala * (x - width / 2)
                imag = escala * (height / 2 - y)
                c = complex(real, imag)
                instability = 1 - self.estabilidad(c, suavizar=True)
                image.putpixel((x, y), int(instability * 255))
                
        return image.show()


In [6]:
# Instanciamos la Clase
set = MandelbrotSet(max_iter=30, threshold=1000)

# Podemos generar y comprobar una matriz compleja de c's si son del conjunto o no. (x,y, threshold)
estabilidad, matriz_compleja_c = set.evaluar_matriz_compleja(10,10,0.1)
#print(matriz_compleja_c)

# Podemos representar gráficamente el conjunto (altura, anchura, escala)
set.plot_set(512,512,0.0095)